In [13]:
from pyagnps import climate

from pathlib import Path

import pandas as pd

In [14]:
clm_annagnps = climate.clm_annagnps_coords(coords=(-89.1, 34.1), start="1980-01-01T1700", end="1980-07-31T0700", date_mode="local")

In [15]:
root_dir = Path().cwd()

df = clm_annagnps.query_nldas2_generate_annagnps_climate_daily(output_filepath= root_dir / "climate_file.csv", float_format='%.2f')

In [16]:
df

,Month,Day,Year,Max_Air_Temperature,Min_Air_Temperature,Precip,Dew_Point,Sky_Cover,Wind_Speed,Wind_Direction,Solar_Radiation,Storm_Type_ID,Potential_ET,Actual_ET,Actual_EI,Input_Units_Code
time,,,,,,,,,,,,,,,,
1980-01-01,1,1,1980,6.300000,2.830000,0.0000,0.788011,None,2.636586,99.671921,0.000000,None,0.1425,None,None,1
1980-01-02,1,2,1980,10.050000,1.010000,0.3343,1.780232,None,0.983013,151.945419,84.589043,None,1.3207,None,None,1
1980-01-03,1,3,1980,7.480000,2.440000,10.3941,4.269094,None,1.936419,273.504150,41.893417,None,0.6817,None,None,1
1980-01-04,1,4,1980,6.410000,1.340000,2.3033,1.615022,None,5.365164,137.888397,107.222420,None,2.5982,None,None,1
1980-01-05,1,5,1980,4.740000,0.250000,0.0000,-1.541558,None,3.231326,155.017639,92.897331,None,2.1424,None,None,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1980-07-27,7,27,1980,30.040001,24.379999,2.2339,23.482895,None,3.180131,105.913399,211.342789,None,5.0532,None,None,1
1980-07-28,7,28,1980,31.820000,23.650000,0.2563,22.864721,None,1.536312,100.881805,294.255005,None,6.7405,None,None,1
1980-07-29,7,29,1980,33.619999,24.440001,0.0000,22.504629,None,2.265506,139.102737,371.034241,None,8.6012,None,None,1


In [150]:
import hvplot.pandas
import hvplot

import holoviews as hv
import panel as pn
from panel.widgets import Select, MultiSelect, MultiChoice

columns = ['Max_Air_Temperature', 'Min_Air_Temperature', 'Precip', 'Dew_Point', 'Wind_Speed', 'Wind_Direction', 'Solar_Radiation', 'Potential_ET']

# dropdown = Select(options=columns, sizing_mode='fixed', height=30)
# def plot_timeseries(df, column):
#     return hv.Curve((df.index, df[column])).opts(responsive=True, height=600)

# interactive_plot = pn.bind(plot_timeseries, df, dropdown)

dropdown = MultiChoice(options=columns)
def plot_multi_timeseries(df, columns):
    if columns:
        overlay_list = []
        for col in columns:
            overlay_list.append(hv.Curve((df.index, df[col]))*hv.Scatter((df.index, df[col])))
        return hv.Overlay(overlay_list).opts(responsive=True, height=800)

interactive_plot = pn.bind(plot_multi_timeseries, df, dropdown)

app = pn.Column(dropdown, interactive_plot, sizing_mode="stretch_width")
# app.show(threaded=True)
app.height = 1000
app
# app.save("climate.html", embed=True)


BokehModel(combine_events=True, render_bundle={'docs_json': {'a16a2e33-562a-4488-82c5-0b3b203c0111': {'version…